In [1]:
import numpy as np
import MetaSet as ms
from MetaSet import dataCaculate as data
from MetaSet import advancedStructure as ad

In [2]:
p=0.4e-6
h=0.6e-6
r=0.12e-6
l=0.3e-6
w=0.1e-6

parameter=[l,w,r]

meta=ad.MetaEngine()
meta.materialSet() # 暂且如此

In [3]:
strClass=4
meta.baseBuild(p)

In [4]:
meta.structureBuild(strClass, parameter, h)

meta.fdtd.save()

In [5]:
meta.dataAcquisition()
print(meta.Ex)
print(meta.Trans)

[0.1564463408534716, -2.162962034319993, -5.101423386265772, -7.615054594970369, -9.067630053729719]
[0.18935915 0.97800712 0.96306406 0.04640321 0.02302144]


In [3]:
P=np.array([0.4e-6])
pp=np.linspace(0.4e-6,0.5e-6,3)